# Soft-Actor-Critic Implementation

Original Paper: https://arxiv.org/pdf/1801.01290.pdf

In [3]:
# Import Python Libraries
import math
import random

# Import OpenAI Gym and NumPy for math
import gym
import numpy as np

# Using PyTorch as DL Framework
import torch

# Imports to display graphs in notebook
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display

# Magic command for inline matplotlib plots
%matplotlib inline

# If using cuda replace "cpu" with "cuda"
device = torch.device("cpu")

## Helper Functions
### ReplayBuffer class for experience replay

In [5]:
# class for storing and sampling from collection of experience tuples (state, action, reward, next_state, done)
# class has capacity, buffer (list) and position.
#       has push method to append a new experience tuple to buffer
#       has sample method that gets a batch of random samples from the buffer 

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

### Normalizing Actions

In [6]:
class NormalizedActions(gym.ActionWrapper):
    def _action(self, action):
        low  = self.action_space.low
        high = self.action_space.high
        
        action = low + (action + 1.0) * 0.5 * (high - low)
        action = np.clip(action, low, high)
        
        return action

    def _reverse_action(self, action):
        low  = self.action_space.low
        high = self.action_space.high
        
        action = 2 * (action - low) / (high - low) - 1
        action = np.clip(action, low, high)
        
        return actions

### Plot reward x frame

In [7]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()

## Defining the Networks

SAC has a state value function V (parameterized by $\psi$), a soft Q-function Q (parameterized by $\theta$), and the policy function $\pi$ (parameterized by $\theta$).

### State Value Network